In [1]:
# importing data and auxiliar functions
%reload_ext autoreload
%load_ext autoreload
%autoreload 2
import saopaulo.sp_grid as gr
import bikescience.distributions as dist
import saopaulo.cycling_potential as cp
import saopaulo.choropleth_folium as choro_folium
import saopaulo.choropleth as choro
from bikescience.slope import plot_slope, plot_slopes, split_route
from shapely.geometry import LineString, Point
import ast
from numpy import nan
import math
from bikescience.arrow import draw_arrow

import folium
import pandas as pd
import geopandas as gpd
from ipywidgets import interact_manual, widgets, fixed
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr
import warnings
import numpy as np
warnings.simplefilter('ignore')

gr.SP_LAT = -23.63
gr.SP_LON = -46.55

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/pedro/.local/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
distritos = [
    'Bela Vista', 'Bom Retiro', 'Cambuci', 'Consolação', 'Liberdade', 
    'República', 'Santa Cecília', 'Sé','Butantã', 'Morumbi', 'Raposo Tavares', 
    'Rio Pequeno', 'Vila Sônia', 'Barra Funda', 'Jaguara', 'Jaguaré', 'Lapa', 
    'Perdizes', 'Vila Leopoldina', 'Alto de Pinheiros', 'Itaim Bibi', 
    'Jardim Paulista', 'Pinheiros','Aricanduva', 'Carrão', 'Vila Formosa',
    'Água Rasa', 'Belém', 'Brás', 'Mooca', 'Pari', 'Tatuapé', 'Artur Alvim',
    'Cangaíba', 'Penha', 'Vila Matilde', 'São Lucas', 'Sapopemba',
    'Vila Prudente','Cidade Tiradentes', 'Ermelino Matarazzo', 'Ponte Rasa',
    'Guaianases', 'Lajeado', 'Itaim Paulista', 'Vila Curuçá', 'Cidade Líder',
    'Itaquera', 'José Bonifácio', 'Parque do Carmo', 'Iguatemi', 'São Mateus',
    'São Rafael', 'Jardim Helena', 'São Miguel', 'Vila Jacuí','Cachoeirinha', 
    'Casa Verde', 'Limão', 'Brasilândia', 'Freguesia do Ó', 'Anhanguera', 
    'Perus', 'Jaraguá', 'Pirituba', 'São Domingos','Jaçanã', 'Tremembé', 
    'Mandaqui', 'Santana', 'Tucuruvi', 'Vila Guilherme', 'Vila Maria', 
    'Vila Medeiros','Cursino', 'Ipiranga', 'Sacomã', 'Jabaquara', 'Moema', 
    'Saúde', 'Vila Mariana', 'Campo Limpo', 'Capão Redondo', 'Vila Andrade', 
    'Cidade Dutra', 'Grajaú', 'Socorro', 'Cidade Ademar', 'Pedreira', 
    'Jardim Ângela', 'Jardim São Luís', 'Marsilac', 'Parelheiros',
    'Campo Belo', 'Campo Grande', 'Santo Amaro']

In [3]:
#read data

#zone_shp = gpd.read_file('../data/sao-paulo/od/shapes/Zonas_2017_region.shp')
#zone_shp.crs = {'init': 'epsg:31983'}  
#zone_shp.to_crs(epsg='4326', inplace=True)
zone_shp = gpd.read_file('../data/sao-paulo/od/shapes/Distritos_2017_region.shp')
zone_shp.crs = {'init': 'epsg:31983'}  
zone_shp.to_crs(epsg='4326', inplace=True)

display(zone_shp)
# converting to km^2
zone_shp['Area_ha_2'] = zone_shp['Area_ha'] / 100
zone_shp['NumeroZona'] = zone_shp['NumeroDist']
zone_shp['NomeZona'] = zone_shp['NomeDistri']
zone_shp['NumeroMuni'] = 0

numeroMuni = []
for _, d in zone_shp.iterrows():
    if d['NomeDistri'] in distritos:
        numeroMuni.append(36)
    else:
        numeroMuni.append(0)
zone_shp['NumeroMuni'] = numeroMuni

# read routes files and join with general trips file
od_trips = pd.read_csv('bases/complete_csv.csv')
# fix FE_VIA with wrong format (because of thousand separator)
od_trips.loc[od_trips['FE_VIA'] >= 10000, 'FE_VIA'] = od_trips.loc[od_trips['FE_VIA'] >= 10000, 'FE_VIA'] / 1000

#od_trips.set_index('ID_ORDEM', inplace=True, drop=False)

od_trips.dropna(inplace = True)

the_grid = gr.create(n=9, west_offset=-0.15, east_offset=0.23, north_offset=0.19, south_offset=-0.46)

,NumeroDist,NomeDistri,Area_ha,geometry
0,1,Água Rasa,715.05,"POLYGON ((-46.55380 -23.56843, -46.55387 -23.5..."
1,2,Alto de Pinheiros,751.26,"POLYGON ((-46.69209 -23.54631, -46.69257 -23.5..."
2,3,Anhanguera,3339.95,"POLYGON ((-46.76082 -23.42733, -46.76088 -23.4..."
3,4,Aricanduva,686.26,"POLYGON ((-46.50153 -23.57941, -46.50154 -23.5..."
4,5,Artur Alvim,653.04,"POLYGON ((-46.47300 -23.54029, -46.47302 -23.5..."
...,...,...,...,...
129,130,São Caetano do Sul,1536.53,"POLYGON ((-46.58317 -23.64088, -46.58326 -23.6..."
130,131,São Lourenço da Serra,18674.34,"POLYGON ((-46.98852 -23.79984, -46.98847 -23.7..."
131,132,Suzano,20666.94,"POLYGON ((-46.38321 -23.61775, -46.38318 -23.6..."
132,133,Taboão da Serra,2043.37,"POLYGON ((-46.81819 -23.63701, -46.82138 -23.6..."


In [4]:
od_trips = gpd.GeoDataFrame(od_trips, crs={'init': 'epsg:4326'},
                 geometry=[LineString(eval(r)) for r in od_trips['route']])

In [5]:
# calculate the number of trips that passes through 
def calculate_trips_zone_intersection (trips, zones, reference):
    
    trips_geometries = list(trips['geometry'])
    trips_expansion = list(trips['FE_VIA'])

    trips_per_zone = [0]*len(zone_shp) # hash to store trips indexed by zones

    progress = 0
    
    for z in range(len(zones)):
        zone = zones.iloc[z]['geometry']
        
        if z / len(zones) >= progress:
            print (round(progress * 100), '%')
            progress += 0.25
            
        for i in range (len (trips_geometries)):
            if reference == 'intersection':
                if (trips_geometries[i].intersects(zone)):
                    trips_per_zone[z] += trips_expansion[i]
            else:
                p = trips_geometries[i].coords[reference]
                if (Point(p[0], p[1]).within(zone)):
                    trips_per_zone[z] += trips_expansion[i]
    return trips_per_zone   

In [6]:
def filter_potential_widgets(btn_text, process_function):
    
    width = '200px'
    layout = widgets.Layout(width = '500px')
    
    im = interact_manual(
        process_function,
        final_potential = widgets.FloatRangeSlider(
            value=[0.8, +1.],
            min=0, max=1., step=0.05,
            description='Final Potential:\t',
            style={'description_width': width},
            layout=layout
        ),
        distance_potential = widgets.FloatRangeSlider(
            value=[0, +1.],
            min=0, max=1., step=0.05,
            description='Distance Potential:\t',
            style={'description_width': width},
            layout=layout
        ),
        inclination_potential = widgets.FloatRangeSlider(
            value=[0, +1.],
            min=0, max=1., step=0.05,
            description='Slope Potential:\t',
            style={'description_width': width},
            layout=layout
        ),
        age_potential = widgets.FloatRangeSlider(
            value=[0, +1.],
            min=0, max=1., step=0.05,
            description='Age Potential:\t',
            style={'description_width': width},
            layout=layout
        ),
        distance = widgets.FloatRangeSlider(
            value=[2, 25],
            min=0, max=25, step=0.05,
            description='Distance (km):\t',
            style={'description_width': width},
            layout=layout
        ),
        
        modal = widgets.Dropdown(options = [('All', None), ('Car', 'car'), ('Subway', 'subway'),
                                            ('Pedestrian', 'pedestrian'), ('Motorcycle', 'motorcycle'),
                                            ('Train', 'train')],
                                 description = 'Modal', 
                                 style={'description_width': width}),
        reference = widgets.Dropdown(options = [('Whole Trip', 'intersection'),('Origin', 0), 
                                                ('Destination', -1)],
                                     description = 'Trip Reference Point:\t', 
                                     style={'description_width': width}),
        
        density = widgets.Checkbox(description = 'Density'),
        debug = widgets.Checkbox(description = 'Show routes'),
        plot_rmsp = widgets.Checkbox(description = 'Plot SPMA'),
        title = widgets.Text(description = 'Map title', style={'description_width': width}),
    )
    im.widget.children[10].description = btn_text
    
def inside_limits (limits, value):
    return value >= limits[0] and value <= limits[1]
    
def filter_potential(final_potential, distance_potential, age_potential, inclination_potential, modal, distance):
    if modal != None:
        print(modal)
        aux = od_trips.loc[od_trips['modal'] == modal]
    else:
        aux = od_trips
    
    aux = aux.loc[aux['age_potential'] >= age_potential[0]]
    aux = aux.loc[aux['age_potential'] <= age_potential[1]]
    aux = aux.loc[aux['distance_potential'] >= distance_potential[0]]
    aux = aux.loc[aux['distance_potential'] <= distance_potential[1]]
    aux = aux.loc[aux['inclination_potential'] >= inclination_potential[0]]
    aux = aux.loc[aux['inclination_potential'] <= inclination_potential[1]]
    aux = aux.loc[aux['final_potential'] >= final_potential[0]]
    aux = aux.loc[aux['final_potential'] <= final_potential[1]]
    aux = aux.loc[aux['length'] >= distance[0] * 1000]
    aux = aux.loc[aux['length'] <= distance[1] * 1000]
    return aux

In [7]:
def plot_potential_trips_per_zone(final_potential, distance_potential, inclination_potential, age_potential, 
                                  modal, reference, 
                                  debug=False, plot_rmsp = False, density = False,
                                  title = None):
    df = filter_potential(final_potential, distance_potential, age_potential, inclination_potential, modal)
    
    global global_df 
    global_df = df
    zones = zone_shp.loc[zone_shp["NumeroMuni"] == 36] if not plot_rmsp else zone_shp
    
    trips_per_zone = calculate_trips_zone_intersection(df, zones, reference)
    
    if density:
        df_total = od_trips if modal == None else od_trips.loc[od_trips['modal'] == modal]
        total_per_zone = calculate_trips_zone_intersection(df_total, zones, reference)
        perc = [0 if total_per_zone[i] == 0 else trips_per_zone[i] / total_per_zone[i] * 100
                for i in range(len(zone_shp))]
        zone_shp['potential_trips'] = perc
        
    else:
        zone_shp['potential_trips'] = trips_per_zone

    # precisa filtrar de novo pra plotar
    zones = zone_shp.loc[zone_shp["NumeroMuni"] == 36] if not plot_rmsp else zone_shp
    
    tooltip_columns = ['NomeZona', 'potential_trips']
    tooltip_aliases = ['District', 'Potential Trips']
    fmap = gr.map_around_sp(the_grid=None,zoom=10,plot_grid=False)
    choro_folium.plot_choropleth(fmap, title, 'YlOrBr', lambda x : x['potential_trips'], 
                          zones, tooltip_columns, tooltip_aliases, plot_rmsp)
    
    if debug:
        #display(df)
        plot_slopes (fmap, df['geometry'], 1000, False)
    fmap.save('maps/' + title + '_potential.html')
    display(fmap)


In [8]:
filter_potential_widgets('Filter trips', plot_potential_trips_per_zone)

interactive(children=(FloatRangeSlider(value=(0.8, 1.0), description='Final Potential:\t', layout=Layout(width…

In [9]:
def build_od_matrix (zones, trips, rounding_trips = True):
    od = {}
    # dicionário com chave (origem, destino)
    # e valor qtde_viagens    
    trips_geometries = list(trips['geometry'])
    trips_expansion = list(trips['FE_VIA'])

    progress = 0
    
    for i in range(len(trips_geometries)):
        if i / len(trips_geometries) >= progress:
            print (round(progress * 100), '%')
            progress += 0.2

        p_o = Point(trips_geometries[i].coords[0]) 
        p_d = Point(trips_geometries[i].coords[-1])
            
        for z_origin in range(len(zones)):
            origin = zones.iloc[z_origin]['geometry']
            if (not p_o.within(origin)): continue
                
            for z_dest in range(len(zones)):
                if z_dest == z_origin and not rounding_trips: continue
                dest = zones.iloc[z_dest]['geometry']
                if (p_d.within(dest)):
                    if (z_origin, z_dest) in od.keys(): 
                        od[(z_origin, z_dest)] += trips_expansion[i]
                    else:
                        od[(z_origin, z_dest)] = trips_expansion[i]
                    break
    return od

def get_centroids (idx_orig, idx_dest, zones):
    orig = zones.iloc[idx_orig]['geometry'].centroid
    dest = zones.iloc[idx_dest]['geometry'].centroid
    reverse = False
    if (idx_orig == idx_dest):
        orig = Point(orig.x + 0.003, orig.y)
        dest = Point(dest.x - 0.003, dest.y)
        reverse = True
    return orig, dest, reverse

def build_flows (flows, zones, zone_identifier = 'NomeDistri'):
    """ flows is a dictionary, returned by function build_od_matrix
        The keys are 2-Tuples (zone_origin, zone_destiny)
        The values are the number of trips between the zones
    """
    df = []
    for f in flows:
        trips = flows[f]
        origin, dest, counterclockwise = get_centroids(f[0], f[1], zones)
        
        df.append([trips, zones.iloc[f[0]][zone_identifier], origin,
                   zones.iloc[f[1]][zone_identifier], dest, counterclockwise])
    df = pd.DataFrame(df, columns = ['trip counts', 'origin_name', 'origin', 'dest_name', 'destination', 'reverse'])
    return df

def flow_map_zones(fmap, od_df, minimum=-1, maximum=-1, show=4, radius=1.0,
                   start_zone_identifier='origin_name', end_zone_identifier='dest_name'):
    if maximum == -1:
        maximum = od_df['trip counts'].max()
    if minimum == -1:
        minimum = maximum / show
        
    total_trips = od_df['trip counts'].sum()

    od_df = od_df[((od_df['trip counts'] >= minimum) & (od_df['trip counts'] <= maximum))]

    for idx, row in od_df.iterrows():
        num_trips = row['trip counts']
        
        weight = math.ceil((num_trips-minimum)/maximum * 10)
        if weight == 0: weight = 1
        
        o1 = row['origin'].y
        o2 = row['origin'].x
        d1 = row['destination'].y
        d2 = row['destination'].x
        
        text_plot = str(round(num_trips)) + '\n'
        text_plot += str(row[start_zone_identifier]) + ' -> ' + str(row[end_zone_identifier]) 
    
        draw_arrow(fmap, o1, o2, d1, d2,
                   text = text_plot,
                   weight = weight,
                   radius_fac = 1.0,
                   loop_arc = row['reverse'])

def plot_flows_widgets(final_potential, distance_potential, inclination_potential, age_potential, distance,
                       modal, grid = False, plot_rmsp = False, rounding_trips = False, title = None, show = 4):
    
    
    trips = filter_potential(final_potential, distance_potential, age_potential, inclination_potential, 
                             modal, distance)
    
    if grid:
        global the_grid
        zones = the_grid.geodataframe()
        zones['NomeDistri'] = zones['i'].astype(str) + (['_'] * len(zones)) + zones['j'].astype(str)
        fmap = gr.map_around_sp(the_grid = the_grid, zoom = 10, plot_grid = True)
    else:
        zones = zone_shp.loc[zone_shp["NumeroMuni"] == 36] if not plot_rmsp else zone_shp
        fmap = gr.map_around_sp(the_grid = None, zoom = 10, plot_grid = False)
        choro.plot_zones(fmap, zone_shp, lambda x : 0, 'white', plot_rmsp = plot_rmsp)
    
    od_dict = build_od_matrix(zones, trips, rounding_trips)
    df = build_flows (od_dict, zones)
        
    flow_map_zones(fmap, df, minimum = -1, maximum = max(df['trip counts']), show = show)
    
    fmap.save('maps/' + title + '_flows.html')
    display(fmap)
    
def filter_flows_widgets():
    
    width = '200px'
    layout = widgets.Layout(width = '500px')
    
    im = interact_manual(
        plot_flows_widgets,
        final_potential = widgets.FloatRangeSlider(
            value=[0.8, +1.],
            min=0, max=1., step=0.05,
            description='Final Potential:\t',
            style={'description_width': width},
            layout=layout
        ),
        distance_potential = widgets.FloatRangeSlider(
            value=[0, +1.],
            min=0, max=1., step=0.05,
            description='Distance Potential:\t',
            style={'description_width': width},
            layout=layout
        ),
        inclination_potential = widgets.FloatRangeSlider(
            value=[0, +1.],
            min=0, max=1., step=0.05,
            description='Slope Potential:\t',
            style={'description_width': width},
            layout=layout
        ),
        age_potential = widgets.FloatRangeSlider(
            value=[0, +1.],
            min=0, max=1., step=0.05,
            description='Age Potential:\t',
            style={'description_width': width},
            layout=layout
        ),
        distance = widgets.FloatRangeSlider(
            value=[2, 25],
            min=0, max=25, step=0.05,
            description='Distance (km):\t',
            style={'description_width': width},
            layout=layout
        ),
        show = widgets.IntSlider(
            value=4,
            min=1, max=25,
            description='Show flows:\t',
            style={'description_width': width},
            layout=layout
        ),
        
        modal = widgets.Dropdown(options = [('All', None), ('Car', 'car'), ('Subway', 'subway'),
                                            ('Pedestrian', 'pedestrian'), ('Motorcycle', 'motorcycle'),
                                            ('Train', 'train')],
                                 description = 'Modal', 
                                 style={'description_width': width}),  
        grid = widgets.Dropdown(options = [('Districts', False), ('Grid', 'grid')],
                                 description = 'Grouped by', 
                                 style={'description_width': width}),
        plot_rmsp = widgets.Checkbox(description = 'Plot SPMA'),
        rounding_trips = widgets.Checkbox(description = 'Rounding trips'),   
        title = widgets.Text(description = 'Map title', style={'description_width': width}),
    )
    im.widget.children[9].description = 'Plot Flows'
    

In [10]:
def define_plot_grid(n, lat_offset, lon_offset):
    global the_grid
    the_grid = gr.create (n, lat_offset[0], lat_offset[1], 
                         lon_offset[0], lon_offset[1])
    fmap = gr.map_around_sp(the_grid = the_grid, zoom = 10)
    choro.plot_zones(fmap, zone_shp, lambda x : 0, 'white', plot_rmsp = False, weight = .5)
    display(fmap)
    
def define_grid_widgets():
    width = '200px'
    layout = widgets.Layout(width = '500px')
    im = interact_manual(
        define_plot_grid,
        n = widgets.IntSlider(
            value=15, min=1, max=50,
            step=1, description='Number of columns',
            style={'description_width': width},
            layout=layout
        ),
        lat_offset = widgets.FloatRangeSlider(
            value=[-.3, .2],
            min=-.5, max=.5, step=0.01,
            description='Latitude offset (x):\t',
            style={'description_width': width},
            layout=layout
        ),
        lon_offset = widgets.FloatRangeSlider(
            value=[-.35, .3],
            min=-.5, max=.5, step=0.01,
            description='Longitude offset (y):\t',
            style={'description_width': width},
            layout=layout
        )
    )
    im.widget.children[3].description = 'Save / Plot Grid'
    
define_grid_widgets()

interactive(children=(IntSlider(value=15, description='Number of columns', layout=Layout(width='500px'), max=5…

In [11]:
filter_flows_widgets()

interactive(children=(FloatRangeSlider(value=(0.8, 1.0), description='Final Potential:\t', layout=Layout(width…